In [20]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

Housing=fetch_california_housing()
print(Housing)


{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n-

In [21]:
data = pd.DataFrame(Housing.data,columns=Housing.feature_names)
data["price"]=pd.DataFrame(Housing.target)
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [22]:
#Separating Dependent and Independent Variables
X=data.drop(columns="price")
y=data["price"]
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [23]:
#Hyperparameter Tuning Function 
def hyperparameter_tuning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,n_jobs=-1,verbose=2,
                             scoring="neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) # Added random_state for reproducibility

from mlflow.models import infer_signature
from urllib.parse import urlparse
signature = infer_signature(X_train, y_train)

# Hyperparameter grids (different ranges for different models)
param_grid_best = {  # Aiming for potentially good performance
    'n_estimators': [200, 300],
    'max_depth': [10, None],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1, 2]
}

param_grid_medium = { # Aiming for a balance
    'n_estimators': [150, 250],
    'max_depth': [7, 9],
    'min_samples_split': [3, 4],
    'min_samples_leaf': [1]
}

param_grid_worst = { # Aiming for potentially poor performance
    'n_estimators': [50, 100],
    'max_depth': [2, 3],  # Restrict max_depth
    'min_samples_split': [5, 10], # Larger splits
    'min_samples_leaf': [5, 10] # Larger leaf sizes
}

# Tracking URI
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

In [25]:
## start the MLFLOW Experiments
def train_and_log_model(X_train, y_train, X_test, y_test, param_grid, model_name):
    with mlflow.start_run(run_name=model_name):  # Give each run a descriptive name
        grid_search = hyperparameter_tuning(X_train=X_train,y_train=y_train,param_grid=param_grid)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Log parameters and metrics
        mlflow.log_params(grid_search.best_params_) # Log all best params at once
        mlflow.log_metric("mse", mse)
        mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Log the model
        if tracking_url_type_store != 'file':
            mlflow.sklearn.log_model(best_model, "model", registered_model_name=model_name)
        else:
            mlflow.sklearn.log_model(best_model, "model", signature=signature)

        print(f"{model_name} Best Hyperparameters: {grid_search.best_params_}")
        print(f"{model_name} Mean Squared Error: {mse}")
        return best_model, mse

# Train and log the three models
model_1, mse_1 = train_and_log_model(X_train, y_train, X_test, y_test, param_grid_best, "RandomForest_Model_1")
model_2, mse_2 = train_and_log_model(X_train, y_train, X_test, y_test, param_grid_medium, "RandomForest_Model_2")
model_3, mse_3 = train_and_log_model(X_train, y_train, X_test, y_test, param_grid_worst, "RandomForest_Model_3")

print("\nModel Comparison:")
print(f"Best Model MSE: {mse_1}")
print(f"Medium Model MSE: {mse_2}")
print(f"Worst Model MSE: {mse_3}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Fitting 3 folds for each of 16 candidates, totalling 48 fits


Successfully registered model 'RandomForest_Model_1'.
2025/02/02 14:07:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest_Model_1, version 1
Created version '1' of model 'RandomForest_Model_1'.


RandomForest_Model_1 Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
RandomForest_Model_1 Mean Squared Error: 0.25091605389908384
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


Successfully registered model 'RandomForest_Model_2'.
2025/02/02 14:11:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest_Model_2, version 1
Created version '1' of model 'RandomForest_Model_2'.


RandomForest_Model_2 Best Hyperparameters: {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 250}
RandomForest_Model_2 Mean Squared Error: 0.3155497501567389
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Fitting 3 folds for each of 16 candidates, totalling 48 fits


Successfully registered model 'RandomForest_Model_3'.
2025/02/02 14:12:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest_Model_3, version 1


RandomForest_Model_3 Best Hyperparameters: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 100}
RandomForest_Model_3 Mean Squared Error: 0.602351853834469

Model Comparison:
Best Model MSE: 0.25091605389908384
Medium Model MSE: 0.3155497501567389
Worst Model MSE: 0.602351853834469


Created version '1' of model 'RandomForest_Model_3'.


In [7]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme